# Create Normal class label from un-label data
### ทำ annotation ของ เสียงปกติ(Normal)มาใช้ เพราะเราจะทำ 3 คลาส osa , hypo ,normal

In [4]:
import os
import json
from glob import glob
import random

# Path to original annotation files
data_path = r"C:\V89\data"
output_path = os.path.join(data_path, "annotation_with_normal")
os.makedirs(output_path, exist_ok=True)

min_normal_sec = 5  # minimum length of normal segment in seconds

ann_files = glob(os.path.join(data_path, "*_annotation.json"))

for ann_file in ann_files:
    with open(ann_file, "r") as f:
        ann = json.load(f)

    record_start = ann["record_start"]
    events = [(e["evnet_start"] - record_start, 
               e["evnet_start"] - record_start + e["event_duration"]) 
              for e in ann["events"]]
    events.sort()

    # Count current labels
    label_counts = {}
    for e in ann["events"]:
        label_counts[e["event_type"]] = label_counts.get(e["event_type"], 0) + 1
    target_normal_count = label_counts.get("hypo", 0) + label_counts.get("osa", 0)

    # Find normal gaps
    normal_segments = []
    last_end = 0.0
    for start, end in events:
        if start - last_end >= min_normal_sec:
            normal_segments.append((last_end, start))
        last_end = max(last_end, end)

    # Use the tail gap if any
    # Note: this assumes we know audio length, so we use the latest event end
    # to define possible tail segment (optional if no audio length)
    if last_end < max([end for _, end in events]):
        audio_end = max([end for _, end in events])
    else:
        audio_end = last_end
    if audio_end - last_end >= min_normal_sec:
        normal_segments.append((last_end, audio_end))

    if not normal_segments:
        print(f"No normal segment found for {os.path.basename(ann_file)}")
        continue

    # Randomly sample normal events
    sampled_normals = []
    while len(sampled_normals) < target_normal_count and normal_segments:
        seg = random.choice(normal_segments)
        start_sec, end_sec = seg
        duration = end_sec - start_sec
        if duration >= min_normal_sec:
            sampled_normals.append({
                "event_type": "normal",
                "evnet_start": start_sec + record_start,
                "event_duration": duration,
                "sleep_stage": "N/A"
            })
            normal_segments.remove(seg)

    # Append to annotation
    ann["events"].extend(sampled_normals)

    # Save new annotation
    save_path = os.path.join(output_path, os.path.basename(ann_file))
    with open(save_path, "w") as f:
        json.dump(ann, f, indent=4)

    print(f"Saved new annotation with normals: {save_path}")


Saved new annotation with normals: C:\V89\data\annotation_with_normal\01_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\02_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\03_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\04_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\05_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\06_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\07_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\08_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\09_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\10_annotation.json
Saved new annotation with normals: C:\V89\data\annotation_with_normal\12_annotation.json
Saved new annotation 